# Проект №3 EDA + Feature Engineering Booking.com

https://www.kaggle.com/code/motoborgrus/proj-3-hotels/notebook?scriptVersionId=103790140
https://www.kaggle.com/code/margaritakr/project-3-booking-com-margaritak/notebook
https://www.kaggle.com/code/ryroman/bookinganalysis

### Какой кейс решаем?
Представим, что работаем дата-сайентистом в компании Booking. Одна из проблем компании — это нечестные отели, которые накручивают себе рейтинг. Одним из способов обнаружения таких отелей является построение модели, которая предсказывает рейтинг отеля. Если предсказания модели сильно отличаются от фактического результата, то, возможно, отель ведёт себя нечестно, и его стоит проверить.

### Наименование столбцов:

* hotel_address — адрес отеля;
* review_date — дата, когда рецензент разместил соответствующий отзыв;
* average_score — средний балл отеля, рассчитанный на основе последнего комментария за последний год;
* hotel_name — название отеля;
* reviewer_nationality — страна рецензента;
* negative_review — отрицательный отзыв, который рецензент дал отелю;
* review_total_negative_word_counts — общее количество слов в отрицательном отзыв;
* positive_review — положительный отзыв, который рецензент дал отелю;
* review_total_positive_word_counts — общее количество слов в положительном отзыве.
* reviewer_score — оценка, которую рецензент поставил отелю на основе своего опыта;
* total_number_of_reviews_reviewer_has_given — количество отзывов, которые рецензенты дали в прошлом;
* total_number_of_reviews — общее количество действительных отзывов об отеле;
* tags — теги, которые рецензент дал отелю;
* days_since_review — количество дней между датой проверки и датой очистки;
* additional_number_of_scoring — есть также некоторые гости, которые просто поставили оценку сервису, но не оставили отзыв. Это число указывает, сколько там действительных оценок без проверки.
* lat — географическая широта отеля;
* lng — географическая долгота отеля.

### Файлы для соревнования

* hotels_train.csv - набор данных для обучения [источник данных](https://github.com/slagovskiy/DST/raw/main/Project-3/data/hotels_train.csv.zip)
* hotels_test.csv - набор данных для оценки качества [источник данных](https://github.com/slagovskiy/DST/raw/main/Project-3/data/hotels_train.csv.zip)
* submission.csv - файл сабмишна в нужном формате [источник данных](https://github.com/slagovskiy/DST/raw/main/Project-3/data/hotels_train.csv.zip)


## 1. Исходные данные

In [124]:
import numpy as np
import pandas as pd
import category_encoders as ce

pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.3f}'.format)
pd.set_option('display.max_rows', 200)

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import os
import re
import string
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
# from textblob import TextBlob, Blobber
# from textblob.sentiments import NaiveBayesAnalyzer
from afinn import Afinn

nltk.download('vader_lexicon', download_dir='.')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import f_classif
from sklearn.ensemble import RandomForestRegressor

# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

[nltk_data] Downloading package vader_lexicon to ....
[nltk_data]   Package vader_lexicon is already up-to-date!


In [125]:
RANDOM_SEED = 42

In [126]:
# DATA_DIR = '/kaggle/input/sf-booking/'
# df_train = pd.read_csv(DATA_DIR + 'hotels_train.csv')
# df_test = pd.read_csv(DATA_DIR + 'hotels_test.csv')
# sample_submission = pd.read_csv(DATA_DIR + 'submission.csv')
# local test only
DATA_DIR = 'data'
df_train = pd.read_csv(DATA_DIR + '/hotels_train.csv.zip')
df_test = pd.read_csv(DATA_DIR + '/hotels_test.csv.zip')
sample_submission = pd.read_csv(DATA_DIR + '/submission.csv.zip')

In [127]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386803 entries, 0 to 386802
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               386803 non-null  object 
 1   additional_number_of_scoring                386803 non-null  int64  
 2   review_date                                 386803 non-null  object 
 3   average_score                               386803 non-null  float64
 4   hotel_name                                  386803 non-null  object 
 5   reviewer_nationality                        386803 non-null  object 
 6   negative_review                             386803 non-null  object 
 7   review_total_negative_word_counts           386803 non-null  int64  
 8   total_number_of_reviews                     386803 non-null  int64  
 9   positive_review                             386803 non-null  object 
 

In [128]:
df_train.head()

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng
0,Stratton Street Mayfair Westminster Borough London W1J 8LT United Kingdom,581,2/19/2016,8.400,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,7,10.000,"[' Leisure trip ', ' Couple ', ' Studio Suite ', ' Stayed 2 nights ', ' Submitted from a mobile device ']",531 day,51.508,-0.144
1,130 134 Southampton Row Camden London WC1B 5AF United Kingdom,299,1/12/2017,8.300,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,14,6.300,"[' Business trip ', ' Couple ', ' Standard Double Room ', ' Stayed 1 night ']",203 day,51.521,-0.123
2,151 bis Rue de Rennes 6th arr 75006 Paris France,32,10/18/2016,8.900,Legend Saint Germain by Elegancia,China,No kettle in room,6,406,No Positive,0,14,7.500,"[' Leisure trip ', ' Solo traveler ', ' Modern Double Room Echo ', ' Stayed 3 nights ', ' Submitted from a mobile device ']",289 day,48.845,2.326
3,216 Avenue Jean Jaures 19th arr 75019 Paris France,34,9/22/2015,7.500,Mercure Paris 19 Philharmonie La Villette,United Kingdom,No Negative,0,607,Friendly staff quiet comfortable room spotlessly clean excellent location,11,8,10.000,"[' Leisure trip ', ' Solo traveler ', ' Standard Room with 1 Double Bed ', ' Stayed 1 night ']",681 day,48.889,2.395
4,Molenwerf 1 1014 AG Amsterdam Netherlands,914,3/5/2016,8.500,Golden Tulip Amsterdam West,Poland,Torn sheets,4,7586,The staff was very friendly and helpful Breakfasts were just extraordinary Room was clean quiet and very spacious,20,10,9.600,"[' Business trip ', ' Couple ', ' Standard Double or Twin Room ', ' Stayed 6 nights ']",516 day,52.386,4.847


In [129]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128935 entries, 0 to 128934
Data columns (total 16 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               128935 non-null  object 
 1   additional_number_of_scoring                128935 non-null  int64  
 2   review_date                                 128935 non-null  object 
 3   average_score                               128935 non-null  float64
 4   hotel_name                                  128935 non-null  object 
 5   reviewer_nationality                        128935 non-null  object 
 6   negative_review                             128935 non-null  object 
 7   review_total_negative_word_counts           128935 non-null  int64  
 8   total_number_of_reviews                     128935 non-null  int64  
 9   positive_review                             128935 non-null  object 
 

In [130]:
df_test.head()

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,tags,days_since_review,lat,lng
0,Via Senigallia 6 20161 Milan Italy,904,7/21/2017,8.100,Hotel Da Vinci,United Kingdom,Would have appreciated a shop in the hotel that sold drinking water etc but not necessity Would recommend if like us you arrive late at night to bring drinks from plane airport as there s no shop nearby There is a minibar though if you want to pay those prices,52,16670,Hotel was great clean friendly staff free breakfast every morning with good selection good wifi connection nice sized room with bath fridge in room Personally loved the fact that the hotel isn t in the city centre but is literally next to a train station that you can easily get to and from the airport city Would definitely stay again,62,1,"[' Leisure trip ', ' Couple ', ' Double Room ', ' Stayed 2 nights ']",13 days,45.533,9.171
1,Arlandaweg 10 Westpoort 1043 EW Amsterdam Netherlands,612,12/12/2016,8.600,Urban Lodge Hotel,Belgium,No tissue paper box was present at the room,10,5018,No Positive,0,7,"[' Leisure trip ', ' Group ', ' Triple Room ', ' Stayed 1 night ']",234 day,52.386,4.834
2,Mallorca 251 Eixample 08008 Barcelona Spain,46,11/26/2015,8.300,Alexandra Barcelona A DoubleTree by Hilton,Sweden,Pillows,3,351,Nice welcoming and service,5,15,"[' Business trip ', ' Solo traveler ', ' Twin Room ', ' Stayed 1 night ', ' Submitted from a mobile device ']",616 day,41.393,2.162
3,Piazza Della Repubblica 17 Central Station 20124 Milan Italy,241,10/17/2015,9.100,Hotel Principe Di Savoia,United States of America,No Negative,0,1543,Everything including the nice upgrade The Hotel has been revamped and what a surprise Love every second of it including in room dining which was excellent,27,9,"[' Leisure trip ', ' Couple ', ' Ambassador Junior Suite ', ' Stayed 1 night ']",656 day,45.480,9.196
4,Singel 303 309 Amsterdam City Center 1012 WJ Amsterdam Netherlands,834,5/16/2016,9.100,Hotel Esther a,United Kingdom,No Negative,0,4687,Lovely hotel v welcoming staff,7,2,"[' Business trip ', ' Solo traveler ', ' Classic Double or Twin Room ', ' Stayed 2 nights ', ' Submitted from a mobile device ']",444 day,52.371,4.889


In [131]:
sample_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128935 entries, 0 to 128934
Data columns (total 2 columns):
 #   Column          Non-Null Count   Dtype
---  ------          --------------   -----
 0   reviewer_score  128935 non-null  int64
 1   id              128935 non-null  int64
dtypes: int64(2)
memory usage: 2.0 MB


In [132]:
sample_submission.head()

,reviewer_score,id
0,1,488440
1,10,274649
2,3,374688
3,10,404352
4,9,451596


Для обработки признаков сольем тренировочный и тестовый датасет в один, для разделения добавим общий признак sample, который будет равен 1 для тренировочных данных и 0 для тестовых. Так же в тестовом датасете создадим целевой признак reviewer_score и заполним его нулями.

In [133]:
df_train['sample'] = 1
df_test['sample'] = 0
df_test['reviewer_score'] = 0

hotels = pd.concat([df_train, df_test], sort=False).reset_index(drop=True)

In [134]:
hotels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515738 entries, 0 to 515737
Data columns (total 18 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               515738 non-null  object 
 1   additional_number_of_scoring                515738 non-null  int64  
 2   review_date                                 515738 non-null  object 
 3   average_score                               515738 non-null  float64
 4   hotel_name                                  515738 non-null  object 
 5   reviewer_nationality                        515738 non-null  object 
 6   negative_review                             515738 non-null  object 
 7   review_total_negative_word_counts           515738 non-null  int64  
 8   total_number_of_reviews                     515738 non-null  int64  
 9   positive_review                             515738 non-null  object 
 

In [135]:
hotels.head()

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng,sample
0,Stratton Street Mayfair Westminster Borough London W1J 8LT United Kingdom,581,2/19/2016,8.400,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,7,10.000,"[' Leisure trip ', ' Couple ', ' Studio Suite ', ' Stayed 2 nights ', ' Submitted from a mobile device ']",531 day,51.508,-0.144,1
1,130 134 Southampton Row Camden London WC1B 5AF United Kingdom,299,1/12/2017,8.300,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,14,6.300,"[' Business trip ', ' Couple ', ' Standard Double Room ', ' Stayed 1 night ']",203 day,51.521,-0.123,1
2,151 bis Rue de Rennes 6th arr 75006 Paris France,32,10/18/2016,8.900,Legend Saint Germain by Elegancia,China,No kettle in room,6,406,No Positive,0,14,7.500,"[' Leisure trip ', ' Solo traveler ', ' Modern Double Room Echo ', ' Stayed 3 nights ', ' Submitted from a mobile device ']",289 day,48.845,2.326,1
3,216 Avenue Jean Jaures 19th arr 75019 Paris France,34,9/22/2015,7.500,Mercure Paris 19 Philharmonie La Villette,United Kingdom,No Negative,0,607,Friendly staff quiet comfortable room spotlessly clean excellent location,11,8,10.000,"[' Leisure trip ', ' Solo traveler ', ' Standard Room with 1 Double Bed ', ' Stayed 1 night ']",681 day,48.889,2.395,1
4,Molenwerf 1 1014 AG Amsterdam Netherlands,914,3/5/2016,8.500,Golden Tulip Amsterdam West,Poland,Torn sheets,4,7586,The staff was very friendly and helpful Breakfasts were just extraordinary Room was clean quiet and very spacious,20,10,9.600,"[' Business trip ', ' Couple ', ' Standard Double or Twin Room ', ' Stayed 6 nights ']",516 day,52.386,4.847,1


In [136]:
hotels.nunique(dropna=False)

hotel_address                                   1493
additional_number_of_scoring                     480
review_date                                      731
average_score                                     34
hotel_name                                      1492
reviewer_nationality                             227
negative_review                               330011
review_total_negative_word_counts                402
total_number_of_reviews                         1142
positive_review                               412601
review_total_positive_word_counts                365
total_number_of_reviews_reviewer_has_given       198
reviewer_score                                    38
tags                                           55242
days_since_review                                731
lat                                             1473
lng                                             1473
sample                                             2
dtype: int64

## 2. Проектирование признаков

Выделим признаки с пропусками в данных

In [ ]:
hotels_nulls = hotels.isnull().sum()
display(hotels_nulls[hotels_nulls > 0])

В признаках lat и lng есть пропуски в данных, это географические координаты, нет смысла подбирать какие то осознанные замены для этих данных, по этому заполним пропуски нулями.

In [ ]:
hotels['lat'] = hotels['lat'].fillna(0, inplace=True)
hotels['lng'] = hotels['lng'].fillna(0, inplace=True)

### 2.1 Название отеля

Первоначально получим количество уникальных отелей по названиям.

In [ ]:
hotels['hotel_name'].nunique()

Выделим отели под одинаковым названием, но с различным адресом и добавим, для уникальности, к названию часть адреса.

In [ ]:
address_list = hotels.pivot_table(
    values = 'hotel_address',
    index = 'hotel_name',
    aggfunc = 'nunique'
)
dupl_hotel = address_list[address_list['hotel_address'] > 1]
display(dupl_hotel)
_temp_list = dupl_hotel[dupl_hotel['hotel_address'] > 1].index.to_list() # через временную переменную быстрее обрабатывается
hotels['hotel_name'] = hotels.apply(lambda x: (x['hotel_name'] + ' in ' + x['hotel_address'].split()[-1])
    if (x['hotel_name'] in _temp_list) else x['hotel_name'], axis=1)
display(hotels['hotel_name'].nunique())

Окончательно получили 1494 уникальных отеля.

### 2.2 Адрес отеля

Из признака hotel_address, извлечём: страну и город.

In [ ]:
hotels['hotel_address'].head(20)

In [ ]:
hotels['hotel_country'] = hotels['hotel_address'].apply(lambda x: x.split()[-1])
hotels['hotel_country'] = hotels['hotel_country'].apply(lambda x:'United Kingdom' if x == 'Kingdom' else x)
country_list = list(hotels['hotel_country'].unique())
display(country_list)

Выделено 6 уникальных стран.

In [ ]:
hotels['hotel_city'] = hotels['hotel_address'].apply(lambda x: x.split()[-2])
hotels['hotel_city'] = hotels['hotel_city'].apply(lambda x: 'London' if x == 'United' else x)
city_list = list(hotels['hotel_city'].unique())
display(city_list)

Так же получено 6 унимальных городов.
В каждой стране представлен только один город, по этому кодировать признак города не имеет смысла, кодируем страну и создаем новый признак.

In [ ]:
hotels['hotel_country_code'] = hotels['hotel_country'].astype('category').cat.codes
hotels.head()

### 2.3 Географические координаты.

Несмотря на то, что данные представлены в числовом виде правильнее их рассматривать как категориальные. Можно постараться сделать расчеты до центра города, или до ближайших станций метро или же до каких то достопримечательностей, но по моему проще избавиться от этих признаков. Если результат нас не удовлетворит, то вернемся к этому моменту.

In [ ]:
hotels = hotels.drop(['lat', 'lng'], axis=1)
hotels.head()

### 2.4 Национальность рецензента.

In [ ]:
hotels['reviewer_nationality'] = hotels['reviewer_nationality'].apply(lambda x: x.strip())

Сделаем признак того, что рецензент является гражданином страны, в котором расположен отель

In [ ]:
hotels['is_citizen'] = hotels['reviewer_nationality'] == hotels['hotel_country']
hotels['is_citizen'] = hotels['is_citizen'].astype('int')

В наборе данных 227 уникальных значений для признака национальности рецензента. Закодируем топ 10, остальных отметим как other.

In [ ]:
top_nations = list(hotels['reviewer_nationality'].value_counts()[:10].index)
hotels['reviewer_nationality'] = hotels['reviewer_nationality'].apply(lambda x: x if x in top_nations else 'other')
hotels['reviewer_nationality_code'] = hotels['reviewer_nationality'].astype('category').cat.codes

Европейские страны по большей части христиане, выделим возможных рецензентов-мусульман.

In [ ]:
muslim_countries = ['Turkey', 'United Arab Emirates', 'Saudi Arabia', 'Qatar', 'Kuwait', 'Oman']
hotels['possible_muslim'] = hotels['reviewer_nationality'].apply(lambda x: 1 if x in muslim_countries else 0)

In [ ]:
hotels.head()

### 2.5 Тэги

Количество тэгов.

In [ ]:
hotels['tags_count'] = hotels['tags'].apply(lambda x: len(re.findall("'[^\'](.+?)[^\']'", x)))

Создадим новые признаки на основании тегов.

In [ ]:
def get_tags(tags_string):
    tags_string = tags_string.replace('[', '').replace(']', '')
    tags_string = tags_string.replace("' ", '').replace(" '", '')
    tags_list = tags_string.split(',')
    tags_list = list(map(lambda x: x.strip(), tags_list))

    reviewer_types = {
        'Solo traveler': 1,
        'Couple': 2,
        'Travelers with friends': 3,
        'Family with young children': 4,
        'Family with older children': 5,
        'Group': 6
    }

    room_types = {
            'without Window': 0,
            'Guestroom': 1,
            'Classic': 2,
            'Single': 3,
            'Standard': 4,
            'Superior': 5,
            'Comfort': 6,
            'Club': 7,
            'Suite': 8,
            'Deluxe': 9,
            'King': 10,
            'Premier': 11
            # 'Double'        # 12
            # 'Other'         # 13
        }


    tag_from_mobile = 0
    tag_with_pet = 0
    tag_leisure_trip = 1
    tag_reviewer_type = 0
    tag_count_nights = 0
    tag_view_room = 0
    tag_room_type = -1

    for _tag in list(tags_list):
        if _tag == 'Submitted from a mobile device':
            tag_from_mobile = 1
            tags_list.remove(_tag)

        if _tag == 'With a pet':
            tag_with_pet = 1
            tags_list.remove(_tag)

        if _tag == 'Business trip':
            tag_leisure_trip = 0
            tags_list.remove(_tag)

        if _tag == 'Leisure trip':
            tag_leisure_trip = 1
            tags_list.remove(_tag)

        if _tag in reviewer_types.keys():
            tag_reviewer_type = reviewer_types[_tag]
            tags_list.remove(_tag)

        if re.fullmatch(r'Stayed\s\d+\snight\w?', _tag):
            tag_count_nights = [float(s) for s in re.findall(r'-?\d+\.?\d*', _tag)][0]
            tags_list.remove(_tag)

        if len(tags_list) > 0:
            if ('View' in tags_list[0]) | ('Panoramic' in tags_list[0]):
                tag_view_room = 1

            for rt in room_types.keys():
                if rt in tags_list[0]:
                    tag_room_type = room_types[rt]

            if 'Double' in tags_list[0] or 'Twin' in tags_list[0]:
                tag_room_type = 12
            if tag_room_type == -1:
                tag_room_type = 13

    return tag_from_mobile, tag_with_pet, tag_leisure_trip, tag_reviewer_type, tag_count_nights, tag_view_room, tag_room_type

hotels[['tag_from_mobile',    # отправлено с мобильного
        'tag_with_pet',       # путешествие с домашними животными
        'tag_leisure_trip',   # путешествие по работе / отдых
        'tag_reviewer_type',  # тип путешествия
        'tag_count_nights',   # количество ночей
        'tag_view_room',      # номер с видом на ...
        'tag_room_type'       # тип номера
        ]
] = pd.DataFrame(hotels['tags'].apply(get_tags).to_list(), index=hotels.index)

In [ ]:
hotels.head()

### 2.6 Дата отзыва

Выделим из даты отзыва месяц.

In [ ]:
hotels['review_date'] = pd.to_datetime(hotels['review_date'])
hotels['review_month'] = hotels['review_date'].dt.month

Выделим день недели

In [ ]:
hotels['day_of_week'] = hotels['review_date'].dt.dayofweek

Выделим месяцы "высокого" сезона.

In [ ]:
display(hotels['review_month'].value_counts())

Выделим топ 4, дальше разброс не сильно большой.

In [ ]:
hotels['high_season'] = hotels['review_month'].apply(lambda x: 1 if x in [8,7,5,10] else 0)

Заменим признак days_since_review на числовой.

In [ ]:
hotels['days_since_review'] = hotels['days_since_review'].apply(lambda x: np.float64(re.findall(r'\d+', x)[0])).convert_dtypes()

In [ ]:
hotels.head()

### 2.7 Отзывы

 Отзывы и их структура поможет разобраться в поставленной задаче лучше всего. Для этого надо изучить отзывы, выделить эмоциональную окраску: негативная, нейтральная, позитивная.

Участники соревнования активно применяют библиотеку VADER, класс SentimentIntensityAnalyzer, не будем от них отставать.

In [ ]:
# Создаём объекты SentimentIntensityAnalyzer
positive = SentimentIntensityAnalyzer()
negative = SentimentIntensityAnalyzer()

# посмотрим как изменится характеристика слова 'nothing':
example = 'nothing'
print(f'Словарь оценок слова "{example}" до преобразований:')
print(negative.polarity_scores(example), '\n')

# добавим контекст:
negative_words = {'small': -1, 'nothing': 10}
positive_words = {'nothing': -10, 'everything': 10}
negative.lexicon.update(negative_words)
positive.lexicon.update(positive_words)

print(f' - после преобразований (для негативного контекста):')
print(negative.polarity_scores(example))
print(f' - после преобразований (для позитивного контекста):')
print(positive.polarity_scores(example))

# фразы "No Negative" тонально будет нести негативную окраску:
example = 'No Negative'
print(f'\nСловарь оценок фразы "{example}":')
print(negative.polarity_scores(example))
example = 'Positive'
print(f'Словарь оценок фразы "{example}":')
print(negative.polarity_scores(example))

# меняем метки отсутствия частей отзыва:
hotels['negative_review_temp'] = hotels['negative_review'].apply(lambda x: x.replace('No Negative', 'Positive'))
hotels['positive_review_temp'] = hotels['positive_review'].apply(lambda x: x.replace('No Positive', 'Negative'))

In [ ]:
%%time

hotels['negative_review_sentiments'] = hotels['negative_review_temp'].apply(lambda x: negative.polarity_scores(x))

hotels['positive_review_sentiments'] = hotels['positive_review_temp'].apply(lambda x: positive.polarity_scores(x))

# формируем новые признаки:
hotels['neg_review_sentiments_neg'] = hotels['negative_review_sentiments'].apply(lambda x: x['neg'])
hotels['neg_review_sentiments_neu'] = hotels['negative_review_sentiments'].apply(lambda x: x['neu'])
hotels['neg_review_sentiments_pos'] = hotels['negative_review_sentiments'].apply(lambda x: x['pos'])
hotels['neg_review_sentiments_compound'] = hotels['negative_review_sentiments'].apply(lambda x: x['compound'])

hotels['pos_review_sentiments_neg'] = hotels['positive_review_sentiments'].apply(lambda x: x['neg'])
hotels['pos_review_sentiments_neu'] = hotels['positive_review_sentiments'].apply(lambda x: x['neu'])
hotels['pos_review_sentiments_pos'] = hotels['positive_review_sentiments'].apply(lambda x: x['pos'])
hotels['pos_review_sentiments_compound'] = hotels['positive_review_sentiments'].apply(lambda x: x['compound'])

## 3. Отбор призаков

### 3.1 Очистка дата-сета от ненужных вспомогательных признаков

In [ ]:
hotels.info()

In [ ]:
hotels_copy = hotels.copy()

In [ ]:
object_columns = [s for s in hotels_copy.columns if hotels_copy[s].dtypes == 'object']
hotels_copy.drop(object_columns, axis = 1, inplace=True)

In [ ]:
hotels_copy.info()

In [ ]:
cat_date_columns = [cd for cd in hotels_copy.columns if ((hotels_copy[cd].dtypes.name == 'category') or (hotels_copy[cd].dtypes == 'datetime64[ns]'))]
hotels_copy.drop(cat_date_columns, axis = 1, inplace=True)

In [ ]:
hotels_copy.info()

### 3.2 Анализ мультиколлинеарности

In [ ]:
cols_num = [
    'review_total_negative_word_counts',
    'review_total_positive_word_counts',
    'total_number_of_reviews_reviewer_has_given',
    'tags_count',
    'tag_count_nights',
    'neg_review_sentiments_neg',
    'neg_review_sentiments_neu',
    'neg_review_sentiments_pos',
    'neg_review_sentiments_compound',
    'pos_review_sentiments_neg',
    'pos_review_sentiments_neu',
    'pos_review_sentiments_pos',
    'pos_review_sentiments_compound'
]

cols_cat = [
    'additional_number_of_scoring',
    'average_score',
    'total_number_of_reviews',
    'days_since_review',
    'hotel_country_code',
    'reviewer_nationality_code',
    'is_citizen',
    'possible_muslim',
    'reviewer_score',
    'is_citizen',
    'possible_muslim',
    'tag_from_mobile',
    'tag_with_pet',
    'tag_leisure_trip',
    'tag_reviewer_type',
    'tag_view_room',
    'tag_room_type',
    'review_month',
    'day_of_week',
    'high_season',
    'review_month',
]

In [ ]:
def show_corr_heatmap(columns_list, title, method):
    fig_, ax_ = plt.subplots(figsize=(15, 12))
    corr = hotels_copy[columns_list].corr(method=method)
    mask = np.triu(np.ones_like(corr, dtype=bool))
    sns.heatmap(corr,
                annot=True,
                linewidths=0.1,
                ax=ax_,
                mask=mask,
                cmap='viridis',
                fmt='.1g')
    ax_.set_title(title, fontsize=18)
    plt.show()

In [ ]:
show_corr_heatmap(cols_num, 'КОРРЕЛЯЦИЯ НЕПРЕРЫВНЫХ ПРИЗНАКОВ', method='pearson')

In [ ]:
drop_columns = ['neg_review_sentiments_pos']

In [ ]:
show_corr_heatmap(cols_cat, 'КОРРЕЛЯЦИЯ КАТЕГОРИАЛЬНЫХ ПРИЗНАКОВ', method='spearman');

In [ ]:
drop_columns += ['is_citizen', 'possible_muslim',]

In [ ]:
hotels_copy.drop(drop_columns, axis = 1, inplace=True)

In [ ]:
hotels_copy.info()

### 3.3 Оценка значимости признаков

Для оценки значимости категориальных признаков будем использовать тест хи-квадрат, для непрерывных признаков — тест ANOVA.

In [ ]:
y = hotels_copy.query('sample == 1').drop(['sample'], axis=1)['reviewer_score'].values
X = hotels_copy.query('sample == 1').drop(['sample', 'reviewer_score'], axis=1)

# визуализируем результат анализа значимости:
imp_num = pd.Series(f_classif(X[X.columns], y)[0], index = X.columns)
imp_num.sort_values(inplace = True)

fig5, ax5 = plt.subplots(figsize=(15, 20))
imp_num.plot(kind = 'barh', color='blue')

In [ ]:
drop_columns = ['day_of_week']

In [ ]:
hotels_copy.drop(drop_columns, axis = 1, inplace=True)

## 4. Обучение модели и получение предсказания

Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [ ]:
train_data = hotels_copy.query('sample == 1').drop(['sample'], axis=1)
test_data = hotels_copy.query('sample == 0').drop(['sample'], axis=1)

y = train_data.reviewer_score.values
X = train_data.drop(['reviewer_score'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

In [ ]:
model = RandomForestRegressor(
    n_estimators=100,
    verbose=1,
    n_jobs=-1,
    random_state=RANDOM_SEED)

In [ ]:
%%time

# обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
# сравниваем предсказанные значения (y_pred) с реальными (y_test),
# метрика Mean Absolute Percentage Error (MAPE) показывает среднюю абсолютную
# процентную ошибку предсказанных значений от фактических:

def mean_absolute_percentage_error(y_tr, y_pr):
    """Получение средней абсолютной ошибки"""
    y_tr, y_pr = np.array(y_tr), np.array(y_pr)
    return np.mean(np.abs((y_tr - y_pr) / y_tr)) * 100

print('MAPE:', round(mean_absolute_percentage_error(y_test, y_pred), 2))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh', color='green');

Получаем предсказание целевой переменной

In [ ]:
test_data = test_data.drop(['reviewer_score'], axis=1)
predict_submission = model.predict(test_data)
sample_submission['reviewer_score'] = predict_submission

#  сохраняем результат:
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

Среди 15 главных для построения модели признаков с итоговым MAPE 11.92 - только два исходных - рейтинг отеля и общее количество отзывов об отеле (несправедливо недооцененный мной признак). Остальные - это признаки, сконструированные в процессе исследования данных. Учитывая, что критерий MAPE на необработанных данных - 14.14%, только при помощи инструментов EDA мне удалось улучшить его примерно на 15,5 процентов. В ряду важнейших признаков есть и дистанция до центра и соотношение количества слов в разных частях отзыва. Но конечно, главную роль сыграли признаки, отвечающие за определение эмоциональной окраски текста.
P.S. Это ноутбук предпоследнего, не самого лучшего моего решения. В последнем варианте, по прежнему не меняя вида и настроек модели, только за счет EDA (работы с признаками) мне удалось улучшить итоговую метрику MAPE до значения 11.55. Учитывая, что это текущее первое место соревнования, считаю, что обнародовать его будет немного не спортивно :D

Если вы в данный момент выполняете свой проект - желаю вам вдохновения и удачи. На все вопросы отвечу с удовольствием!